In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("NetflixAnalysis").getOrCreate()

23/12/08 09:52:45 WARN Utils: Your hostname, Srivenis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.201 instead (on interface en0)
23/12/08 09:52:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/08 09:52:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load data into a DataFrame
file_path = "/Users/srivenikunduru/Downloads/NOSQL_Project/final_data.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
# Create a temporary view for running SQL queries
df.createOrReplaceTempView("netflix_data")


In [5]:
# Perform  analysis on type
spark.sql("SELECT type, COUNT(*) AS count FROM netflix_data GROUP BY type").show()

+--------------------+-----+
|                type|count|
+--------------------+-----+
|Mature Audience (TV)|    1|
|             TV Show| 1757|
|               Movie| 3299|
+--------------------+-----+



In [6]:
# Defining a function to find "Avengers: Infinity War"
avengers_query = spark.sql("SELECT * FROM netflix_data WHERE Title = 'Avengers: Infinity War'")
avengers_query.show()

+--------+-----+--------------------+--------------------+--------------------+-------------+----------+---------------+------------+------+--------+--------------------+--------------------+--------------------+
| show_id| type|               Title|            director|                cast|      country|date_added|date_added_year|release_year|rating|duration|           listed_in|         description|         rating_ABBR|
+--------+-----+--------------------+--------------------+--------------------+-------------+----------+---------------+------------+------+--------+--------------------+--------------------+--------------------+
|80219127|Movie|Avengers: Infinit...|Anthony Russo, Jo...|Robert Downey Jr....|United States| 25-Dec-18|           2018|        2018| PG-13| 150 min|Action & Adventur...|Superheroes amass...|Parents Strongly ...|
+--------+-----+--------------------+--------------------+--------------------+-------------+----------+---------------+------------+------+--------

In [7]:
# query utilizing the $regex operator to identify records that contain particular words within the 'Country' column.
countries_to_match = ['India', 'Canada']
country_query = spark.sql(f"SELECT * FROM netflix_data WHERE country RLIKE '(\\b{'|'.join(countries_to_match)}\\b)'")
country_query.show()

+-------+----+-----+--------+----+-------+----------+---------------+------------+------+--------+---------+-----------+-----------+
|show_id|type|Title|director|cast|country|date_added|date_added_year|release_year|rating|duration|listed_in|description|rating_ABBR|
+-------+----+-----+--------+----+-------+----------+---------------+------------+------+--------+---------+-----------+-----------+
+-------+----+-----+--------+----+-------+----------+---------------+------------+------+--------+---------+-----------+-----------+



In [8]:
# Building a regex patterns to search for partial words in 'description' field
partial_words_to_search_relationship = ['relationship', 'romance', 'marriage', 'love']
partial_words_to_search_dystopian = ['dystopian', 'thriller']

relationship_query = spark.sql(f"SELECT * FROM netflix_data WHERE description RLIKE '(\\b{'|'.join(partial_words_to_search_relationship)}\\b)'")
dystopian_query = spark.sql(f"SELECT * FROM netflix_data WHERE description RLIKE '(\\b{'|'.join(partial_words_to_search_dystopian)}\\b)'")

relationship_query.show()
dystopian_query.show()


+--------+-------+--------------------+--------------------+--------------------+--------------------+----------+---------------+------------+------+--------+--------------------+--------------------+--------------------+
| show_id|   type|               Title|            director|                cast|             country|date_added|date_added_year|release_year|rating|duration|           listed_in|         description|         rating_ABBR|
+--------+-------+--------------------+--------------------+--------------------+--------------------+----------+---------------+------------+------+--------+--------------------+--------------------+--------------------+
|80057969|  Movie|                Love|          Gaspar Noé|Karl Glusman, Kla...|     France, Belgium| 08-Sep-17|           2017|        2015|    NR| 135 min|Cult Movies, Dram...|A man in an unsat...|           Not Rated|
|80239639|  Movie|Sierra Burgess Is...|         Ian Samuels|Shannon Purser, K...|       United States| 07-Sep-18